Service Mesh - Istio
--------------------

![](https://istio.io/docs/concepts/what-is-istio/arch.svg)

Quelle: istio.io

- - -

Cloud-Plattformen bieten den Unternehmen, die sie verwenden, zahlreiche Vorteile.

Dazu müssen Entwickler das Architekturmuster Microservices verwenden, was dazu führt das schnell Unterschiedliche Versionen von Microservices parallel betrieben werden müssen.

Mit den vielfältigen Funktionen von [Istio](http://istio.io) kann man eine verteilte Microservice-Architektur erfolgreich und effizient ausführen und auf einheitliche Weise Microservices absichern, verbinden und überwachen.

Es ist ein vollständiger Open-Source-Service-Mesh, das auf vorhandenen verteilten Anwendungen aufbaut.

***
### Was ist ein Service-Mesh?

Der Begriff Service-Mesh beschreibt das Netzwerk von Microservices und die Wechselwirkungen zwischen ihnen. 

Je grösser und komplexer ein Service-Mesh wird, desto schwieriger ist es, es zu verstehen und zu verwalten. 

Zu den Anforderungen zählen Erkennung, Lastenausgleich, Fehlerbehebung, Metriken und Überwachung. 

Ein Service-Mesh hat häufig komplexere betriebliche Anforderungen, wie A/B-Tests (Bewertung zweier Varianten), Canary-Rollouts, Beschränkungen, Zugangskontrolle und End-to-End-Authentifizierung.

Weitere Informationen: [Istio Homepage](https://istio.io/docs/concepts/what-is-istio/)

***
### Installation

Istio ist in der [duk-demo](https://github.com/mc-b/duk-demo) Umgebung bereits installiert. Ansonsten siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/istio).

***
### Bookinfo Beispiel

Die Beispielanwendung zeigt Informationen zu einem Buch an, ähnlich einem einzelnen Katalogeintrag eines Online-Buchladens. Auf der Seite werden eine Beschreibung des Buches, Buchdetails (ISBN, Seitenzahl usw.) und einige Buchbesprechungen angezeigt.

Für Details siehe [Bookinfo Application](https://istio.io/docs/examples/bookinfo/) und den [Sourcecode auf Github](https://github.com/istio/istio/tree/master/samples/bookinfo). 

Die Beispielanwendung ist Bestandteil der Istio Sourcen. Deshalb müssen wir diese zuerst downloaden, dann erstellen wir einen eigenen Namespace wo istio aktiviert ist und darin die Bookinfo Beispielanwendung.


In [1]:
! curl -s -L https://git.io/getLatestIstio | ISTIO_VERSION=1.3.3 sh -
! ls -l istio-1.3.3/samples/bookinfo

                                 Dload  Upload   Total   Spent    Left  Speed
100   614    0   614    0     0   1987      0 --:--:-- --:--:-- --:--:--  1987
100 36.3M  100 36.3M    0     0  2666k      0  0:00:13  0:00:13 --:--:-- 2687k
Istio 1.3.3 Download Complete!

Istio has been successfully downloaded into the istio-1.3.3 folder on your system.

Next Steps:
See https://istio.io/docs/setup/kubernetes/install/ to add Istio to your Kubernetes cluster.

To configure the istioctl client tool for your workstation,
add the /home/jovyan/work/demo/istio-1.3.3/bin directory to your environment path variable with:
	 export PATH="$PATH:/home/jovyan/work/demo/istio-1.3.3/bin"

Begin the Istio pre-installation verification check by running:
	 istioctl verify-install 

Need more information? Visit https://istio.io/docs/setup/kubernetes/install/ 
total 36
drwxrwxrwx 1 jovyan 1000 12288 Oct 11 00:56 networking
drwxrwxrwx 1 jovyan 1000     0 Oct 11 00:56 platform
drwxrwxrwx 1 jovyan 1000  8192 Oct 1

In [2]:
! kubectl create namespace bookinfo
! kubectl label namespace bookinfo istio-injection=enabled
! kubectl apply -f istio-1.3.3/samples/bookinfo/platform/kube/bookinfo.yaml -n bookinfo
! kubectl apply -f istio-1.3.3/samples/bookinfo/networking/bookinfo-gateway.yaml -n bookinfo
! kubectl apply -f istio-1.3.3/samples/bookinfo/networking/destination-rule-all.yaml -n bookinfo

namespace/bookinfo created
namespace/bookinfo labeled
service/details created
serviceaccount/bookinfo-details created
deployment.apps/details-v1 created
service/ratings created
serviceaccount/bookinfo-ratings created
deployment.apps/ratings-v1 created
service/reviews created
serviceaccount/bookinfo-reviews created
deployment.apps/reviews-v1 created
deployment.apps/reviews-v2 created
deployment.apps/reviews-v3 created
service/productpage created
serviceaccount/bookinfo-productpage created
deployment.apps/productpage-v1 created
gateway.networking.istio.io/bookinfo-gateway created
virtualservice.networking.istio.io/bookinfo created
destinationrule.networking.istio.io/productpage created
destinationrule.networking.istio.io/reviews created
destinationrule.networking.istio.io/ratings created
destinationrule.networking.istio.io/details created


Neben Pods, Services etc. wurden auch weitere Ressourcen erstellt wie:
* [Gateway](https://istio.io/docs/concepts/traffic-management/#gateways) - für den Zugriff von ausserhalb des Clusters
* [Virtual Service](https://istio.io/docs/concepts/traffic-management/#virtual-services) - mit den URL Paths 
* [Routing Rule](https://istio.io/docs/concepts/traffic-management/#routing-rules) - mit den Regeln was wo weitergeleitet wird.


In [3]:
! kubectl get all,gw,vs,dr -n bookinfo

NAME                                  READY   STATUS    RESTARTS   AGE
pod/details-v1-74f858558f-n6bng       2/2     Running   0          21s
pod/productpage-v1-8554d58bff-t7827   2/2     Running   0          19s
pod/ratings-v1-7855f5bcb9-59z4n       2/2     Running   0          21s
pod/reviews-v1-59fd8b965b-gnbn6       2/2     Running   0          20s
pod/reviews-v2-d6cfdb7d6-69pnz        2/2     Running   0          20s
pod/reviews-v3-75699b5cfb-l5n8c       2/2     Running   0          20s

NAME                  TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
service/details       ClusterIP   10.101.81.241    <none>        9080/TCP   22s
service/productpage   ClusterIP   10.109.217.115   <none>        9080/TCP   21s
service/ratings       ClusterIP   10.102.23.33     <none>        9080/TCP   22s
service/reviews       ClusterIP   10.107.178.86    <none>        9080/TCP   21s

NAME                             READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/details-v1     

In [4]:
! kubectl get gw,vs,dr -n bookinfo -o yaml

apiVersion: v1
items:
- apiVersion: networking.istio.io/v1alpha3
  kind: Gateway
  metadata:
    annotations:
      kubectl.kubernetes.io/last-applied-configuration: |
        {"apiVersion":"networking.istio.io/v1alpha3","kind":"Gateway","metadata":{"annotations":{},"name":"bookinfo-gateway","namespace":"bookinfo"},"spec":{"selector":{"istio":"ingressgateway"},"servers":[{"hosts":["*"],"port":{"name":"http","number":80,"protocol":"HTTP"}}]}}
    creationTimestamp: "2019-10-28T18:18:43Z"
    generation: 1
    name: bookinfo-gateway
    namespace: bookinfo
    resourceVersion: "47521"
    selfLink: /apis/networking.istio.io/v1alpha3/namespaces/bookinfo/gateways/bookinfo-gateway
    uid: d8607d90-94bf-47e7-b7f1-fa4d5cb89cf8
  spec:
    selector:
      istio: ingressgateway
    servers:
    - hosts:
      - '*'
      port:
        name: http
        number: 80
        protocol: HTTP
- apiVersion: networking.istio.io/v1alpha3
  kind: VirtualService
  metadata:
    annotations:
      kubectl

![](https://istio.io/docs/examples/bookinfo/noistio.svg)

Quelle: istio.io

- - - 

Schauen wir uns aber zuerst die Applikation an. 

Die Bookinfo-Anwendung ist in vier separate Microservices unterteilt:

* productpage: Der productpage Microservice ruft die Microservices details und reviews auf, um die Seite zu füllen.
* details: Der details Microservice enthält Buchinformationen.
* reviews: Der reviews Microservice enthält Buchbesprechungen. Es ruft auch den ratings Microservice auf.
* ratings: Der ratings Microservice enthält Buchranginformationen, die einer Buchbesprechung beigefügt sind.

Es gibt 3 Versionen des reviews Microservice:
* Version v1 ruft den ratings Microservice nicht auf.
* Version v2 ruft den ratings Microservice auf und zeigt jede Bewertung als 1 bis 5 schwarze Sterne an.
* Version v3 ruft den ratings Microservice auf und zeigt jede Bewertung als 1 bis 5 rote Sterne an.

Diese Anwendung sind in mehreren Programmiersprache implementiert.

Die Microservices enthalten keine Abhängigkeiten und wissen auch nichts von Istio. Sie wurden auch nicht speziell Angepasst um mit Istio zu funktionieren.

Das User Interface kann wie folgt angewählt werden:

In [5]:
! echo "$(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/31380/g")/productpage"

http://192.168.137.100:31380/productpage


Wählen wir die Seite mehrmals an, wird abwechselnd einer der drei Version vom Microservice `reviews` aufgerufen.

Um gezielt:

* nur eine Version aufzurufen, Version 1 oder 3
* Analog eines Canary Rollout 50/50 Prozent Version 2 und 3 aufzurufen

stehen vorbereitete YAML Dateien zu Verfügung. Führen Sie diese jeweils einzeln aus und Testen im UI den Effekt. Dabei kann 1 - 2 Minuten dauern, bis die Regeln aktiv werden.

In [6]:
! kubectl apply -f istio-1.3.3/samples/bookinfo/networking/virtual-service-all-v1.yaml -n bookinfo

virtualservice.networking.istio.io/productpage created
virtualservice.networking.istio.io/reviews created
virtualservice.networking.istio.io/ratings created
virtualservice.networking.istio.io/details created


In [7]:
! kubectl apply -f istio-1.3.3/samples/bookinfo/networking/virtual-service-reviews-v3.yaml -n bookinfo

virtualservice.networking.istio.io/reviews configured


In [8]:
! kubectl apply -f istio-1.3.3/samples/bookinfo/networking/virtual-service-reviews-v2-v3.yaml -n bookinfo

virtualservice.networking.istio.io/reviews configured


*** 
### Tools

#### Kiali — Observability

[Kiali](https://www.kiali.io/) ist eine Web UI für Istio. Es hilft Ihnen, die Struktur Ihres Servicenetzes und deren Topologie zu verstehen.
  
#### Jaeger — Tracing

Die Ablaufinformationen für Kiali werden durch das verteiltes Tracing-System [Jaeger](https://www.jaegertracing.io/) bereitgestellt.

Beide Tools sind jedoch nur auf dem Cluster zugänglich. Deshalb müssen wir diese zuerst freischalten bzw. gegen NodePort öffnen:

In [9]:
! kubectl get service -n istio-system -l app=kiali -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! kubectl get service jaeger-query -n istio-system -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -

service/kiali configured
service/jaeger-query configured


Sind Sie freigeschaltet, können wir Sie wie andere Services via Cluster-IP und Port erreichen:

In [10]:
! echo "Kaili  UI: " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service -n istio-system -l app=kiali -o yaml -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')/g")
! echo "Jeager UI: " $(kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service jaeger-query -n istio-system -o=jsonpath='{ .spec.ports[0].nodePort }')/g")  

Kaili  UI:  http://192.168.137.100:32183
Jeager UI:  http://192.168.137.100:32461


Aufräumen

In [11]:
! kubectl delete namespace bookinfo

namespace "bookinfo" deleted


***

Weitere Beispiele siehe Projekt [duk](https://github.com/mc-b/duk/tree/master/istio).
